# 使用🤗 PEFT加载适配器

参数高效微调（PEFT）方法在微调期间冻结预训练模型参数，并在其顶部添加少量可训练参数（适配器）。适配器经过训练以学习特定任务的信息。这种方法已被证明在产生与完全微调模型可比的结果的同时，具有非常高效的内存使用和较低的计算使用。

使用PEFT训练的适配器通常比完整模型小一个数量级，这使得共享、存储和加载它们变得更加方便。

![](https://image.rarelimiting.com/PEFT-hub-screenshot.png)


“存储在Hub上的OPTForCausalLM模型的适配器权重仅约6MB，而完整模型权重的大小可能约为700MB。”


## 设置

开始安装🤗 PEFT：


```bash
pip install peft
```


如果您想尝试全新的功能，您可能会对从源代码安装该库感兴趣：


```bash
pip install git+https://github.com/huggingface/peft.git
```


## 支持的PEFT模型

🤗 Transformers原生支持一些PEFT方法，这意味着您可以加载存储在本地或Hub上的适配器权重，并使用几行代码轻松运行或训练它们。支持以下方法：

- 低秩适配器（Low Rank Adapters）
- IA3
- AdaLoRA

如果您想使用其他PEFT方法，如提示学习或提示调整，或者了解🤗 PEFT库的一般信息，请参阅文档。

## 加载PEFT适配器

要从🤗 Transformers加载和使用PEFT适配器模型，请确保Hub存储库或本地目录包含一个adapter_config.json文件和适配器权重，如上面的示例图所示。然后，您可以使用AutoModelFor类加载PEFT适配器模型。例如，要加载用于因果语言建模的PEFT适配器模型：

- 指定PEFT模型id
- 将其传递给AutoModelForCausalLM类

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "ybelkada/opt-350m-lora"
model = AutoModelForCausalLM.from_pretrained(peft_model_id)

您可以使用AutoModelFor类或基本模型类（如OPTForCausalLM或LlamaForCausalLM）加载PEFT适配器。

您还可以通过调用load_adapter方法加载PEFT适配器：

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "facebook/opt-350m"
peft_model_id = "ybelkada/opt-350m-lora"

model = AutoModelForCausalLM.from_pretrained(model_id)
model.load_adapter(peft_model_id)

## 加载8位或4位精度

bitsandbytes集成支持8位和4位精度数据类型，这对于加载大型模型非常有用，因为它可以节省内存（请参阅bitsandbytes集成指南以了解更多信息）。将load_in_8bit或load_in_4bit参数添加到from_pretrained()中，并设置device_map="auto"以有效地将模型分配到您的硬件：

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "ybelkada/opt-350m-lora"
model = AutoModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", load_in_8bit=True)

## 添加新适配器

只要新适配器与当前适配器的类型相同，您可以使用~peft.PeftModel.add_adapter向具有现有适配器的模型添加新适配器。例如，如果您已将现有的LoRA适配器附加到模型上：

In [ ]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer
from peft import LoraConfig

model_id = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_id)

lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    init_lora_weights=False
)

model.add_adapter(lora_config, adapter_name="adapter_1")

要添加一个新的适配器：

In [ ]:
# attach new adapter with same config
model.add_adapter(lora_config, adapter_name="adapter_2")

现在您可以使用~peft.PeftModel.set_adapter来设置要使用的适配器：

In [ ]:
# use adapter_1
model.set_adapter("adapter_1")
output = model.generate(**inputs)
print(tokenizer.decode(output_disabled[0], skip_special_tokens=True))

# use adapter_2
model.set_adapter("adapter_2")
output_enabled = model.generate(**inputs)
print(tokenizer.decode(output_enabled[0], skip_special_tokens=True))

## 启用和禁用适配器

一旦您向模型添加了适配器，您可以启用或禁用适配器模块。要启用适配器模块：

In [ ]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer
from peft import PeftConfig

model_id = "facebook/opt-350m"
adapter_model_id = "ybelkada/opt-350m-lora"
tokenizer = AutoTokenizer.from_pretrained(model_id)
text = "Hello"
inputs = tokenizer(text, return_tensors="pt")

model = AutoModelForCausalLM.from_pretrained(model_id)
peft_config = PeftConfig.from_pretrained(adapter_model_id)

# to initiate with random weights
peft_config.init_lora_weights = False

model.add_adapter(peft_config)
model.enable_adapters()
output = model.generate(**inputs)

要禁用适配器模块：

In [ ]:
model.disable_adapters()
output = model.generate(**inputs)

## 训练一个PEFT适配器

PEFT适配器受Trainer类支持，这样您就可以为特定用例训练一个适配器。只需要添加几行代码。例如，要训练一个LoRA适配器：

> 如果您对使用Trainer微调模型不熟悉，请查看微调预训练模型教程。

1. 使用任务类型和超参数定义您的适配器配置（有关超参数的详细信息，请参阅~peft.LoraConfig）。

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

2. 将适配器添加到模型。

In [ ]:
model.add_adapter(peft_config)

3. 现在您可以将模型传递给Trainer！

In [ ]:
trainer = Trainer(model=model, ...)
trainer.train()

要保存您训练的适配器并加载回来：

In [ ]:
model.save_pretrained(save_dir)
model = AutoModelForCausalLM.from_pretrained(save_dir)

## 向PEFT适配器添加额外可训练层

您还可以通过在PEFT配置中传递modules_to_save来在已附加适配器的模型顶部微调额外的可训练适配器。例如，如果您想在具有LoRA适配器的模型顶部微调lm_head：

In [ ]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer
from peft import LoraConfig

model_id = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_id)

lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    modules_to_save=["lm_head"],
)

model.add_adapter(lora_config)